In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from google.cloud import storage
#from pyspark.sql.functions import col, mean, stddev, expr, date_format, count, avg
from pyspark.sql import DataFrame
import pandas as pd
#import matplotlib.pyplot as plt

In [2]:
# Create SparkContext
sc = SparkContext.getOrCreate()

# Set the log level to ERROR to suppress INFO messages
sc.setLogLevel("ERROR")

In [3]:
#fix the formating of the shows, so they don't overlap.
def hscroll(activate=True):
  """activate/deactivate horizontal scrolling for wide output cells"""
  from IPython.display import display, HTML
  style = ('pre-wrap','pre')[activate] # select white-space style
  display(HTML("<style>pre {white-space: %s !important}</style>" % style))
hscroll()

In [4]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("Combine Parquet Files") \
    .getOrCreate()

# Create a client object that points to GCS
storage_client = storage.Client()

# Get a list of the 'blobs' (objects or files) in the bucket
blobs = storage_client.list_blobs('my-bigdata-project-cm', prefix="aggregated/")

# Initialize an empty DataFrame to store the combined data
combined_df = None

# Run through the my-bigdata-project-cm bucket.
for blob in blobs:
    file_path = f'gs://my-bigdata-project-cm/{blob.name}'
    
    # Exclude the file title.
    if not blob.name.endswith('.parquet') or blob.name == 'aggregated/':
        continue
        
    try:
        # Read Parquet file from Google Cloud Storage
        sdf = spark.read.parquet(file_path)
        
        # Combine dataframes
        if combined_df is None:
            combined_df = sdf
        else:
            combined_df = combined_df.union(sdf)
    
    except Exception as e:
        print(f"An error occurred on {blob.name}:", str(e))   
        
        # Check if the error is related to SparkContext
        if "Cannot call methods on a stopped SparkContext" in str(e):
            print("SparkContext error occurred. Stopping the program.")
            break  # Exit the loop

# Write the combined DataFrame to a single Parquet file on local system
#combined_df.write.parquet("C:/Users/ASCStudent/Desktop/classes/CIS 4130/project/m5/combined_data.parquet", mode="overwrite")
#combined_df.write.parquet("combined_data.parquet", mode="overwrite")

# Stop the SparkSession
#spark.stop()

24/05/04 01:04:33 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Py4JJavaError: An error occurred while calling o8890.parquet.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "C"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3586)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3609)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:175)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3713)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3664)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:558)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWritingFileFormat(DataSource.scala:454)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWriting(DataSource.scala:530)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:388)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:361)
	at or

In [8]:
combined_df.first()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o8888.limit

In [9]:
combined_df.write.parquet("combined_data.parquet", mode="overwrite")

ConnectionRefusedError: [Errno 111] Connection refused

# Write the combined DataFrame to a local CSV file
combined_df.to_csv('C:/Users/ASCStudent/Desktop/classes/CIS 4130/project/m5/combined_data.csv', index=False)